In [ ]:
import d3rlpy
import os
import pandas as pd
import numpy as np
from d3rlpy.dataset import MDPDataset

In [ ]:
base_dir = os.path.expanduser("./Data") # path of the datasets

def build_dataset_from_csv(path, episode_len=48):   # process 1-6 datasets
    df = pd.read_csv(path)

    s1 = df.iloc[:, 3].values.astype(np.float32)
    s2 = df.iloc[:, 4].values.astype(np.float32)
    s3_raw = df.iloc[:, 6].values
    s3 = np.where(s3_raw == 0, 0, 1).astype(np.float32)

    states = np.stack([s1, s2, s3], axis=1)

    actions_raw = df.iloc[:, 5].values
    actions = np.where(actions_raw == 0, 0, 1).astype(np.int64)

    s1 = df.iloc[:, 3].values.astype(np.float32)
    rewards = (
        -( (s1 > 140) * np.abs(s1 - 140) ** 1.10
         + (s1 < 80)  * (s1 - 80) ** 2 ) / 30.0
    ).astype(np.float32)

    rews = (rewards+40.833332)/40.833332

    n = len(df)
    terminals = np.zeros(n, dtype=np.float32)
    terminals[episode_len - 1 :: episode_len] = 1.0

    return MDPDataset(
        observations=states,
        actions=actions,
        rewards=rews,
        terminals=terminals
    )

datasets = {}
for i in range(1, 7):
    name = f"pat{i}"
    csv_path = os.path.join(base_dir, f"{name}.csv")
    datasets[name] = build_dataset_from_csv(csv_path)

    print(f"{name} -> {datasets[name].size()} transitions, "
          f"{len(datasets[name].episodes)} episodes")


    h5_path = os.path.join(base_dir, f"{name}_binary_meal.h5")
    datasets[name].dump(h5_path)



data_pat1 = datasets["pat1"]
data_pat2 = datasets["pat2"]
data_pat3 = datasets["pat3"]
data_pat4 = datasets["pat4"]
data_pat5 = datasets["pat5"]
data_pat6 = datasets["pat6"]

In [11]:
from d3rlpy.preprocessing import StandardObservationScaler
import d3rlpy
from d3rlpy.models.encoders import VectorEncoderFactory
from d3rlpy.metrics import TDErrorEvaluator
from d3rlpy.metrics import InitialStateValueEstimationEvaluator

In [ ]:
def train_sac(MDP_data, gamma = 0.9, n_steps = 30000):

    encoder_factory_1 = VectorEncoderFactory(hidden_units=[128, 128], activation='relu')
    encoder_factory_2 = VectorEncoderFactory(hidden_units=[128, 128], activation='relu')
    sac_beh = d3rlpy.algos.DiscreteSACConfig(observation_scaler=StandardObservationScaler(),batch_size=64, gamma=gamma,
                                             critic_encoder_factory=encoder_factory_1,
                                             actor_encoder_factory=encoder_factory_2,
                                             target_update_interval=1000).create(device="cuda:0")
    sac_beh.build_with_dataset(MDP_data)

    sac_beh.fit(MDP_data,
        n_steps=n_steps,
        n_steps_per_epoch=2000,
        show_progress=False,
        evaluators={
            'TD': TDErrorEvaluator(),
            'Value': InitialStateValueEstimationEvaluator(),
        },
    )

    return sac_beh

In [ ]:
datasets = {f"pat{i}": globals()[f"data_pat{i}"] for i in range(1, 7)}

In [ ]:
d3rlpy.seed(123)
base_dir = os.path.expanduser(
    "./Models"
)
#os.makedirs(base_dir, exist_ok=True)

for i in range(1, 7):
    name = f"pat{i}"
    print(f"Training SAC for {name} ...")

    mdp = datasets[name]              
    model = train_sac(mdp, gamma=0.9) 

    globals()[f"sac_beh_{i}"] = model


    model_path = os.path.join(base_dir, f"sac_model_pat{i}.d3")
    model.save(model_path)

    print(f"Saved sac_beh_{i} to {model_path}")

In [ ]:
d3rlpy.seed(123)
model4 = train_sac(datasets[f"pat{4}"], gamma=0.9,n_steps = 20000) 

globals()[f"sac_beh_{4}"] = model4

model_path = os.path.join(base_dir, f"sac_model_pat{4}.d3")
model.save(model_path)

print(f"Saved sac_beh_{4} to {model_path}")

In [ ]:
d3rlpy.seed(123)
model6 = train_sac(datasets[f"pat{6}"], gamma=0.9,n_steps = 18000) 

globals()[f"sac_beh_{6}"] = model6

model_path = os.path.join(base_dir, f"sac_model_pat{6}.d3")
model.save(model_path)

print(f"Saved sac_beh_{6} to {model_path}")